In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_graphics as tfg
from tensorflow_graphics.nn.loss import chamfer_distance
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
from matplotlib import pyplot as p

import torch
from chamferdist import ChamferDistance

# %load_ext autoreload
# %autoreload 2
# %autosave 180

2024-09-30 15:25:48.605403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 15:25:48.702208: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 15:25:49.026967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/lib:
2024-09-30 15:25:49.027032: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7:

In [70]:
# nerf_pc = np.load("lidar_nerf_demo/generated_pc_at_start.npy")
# real_pc = np.load("lidar_nerf_demo/real_pc_at_start.npy")
# nerf_pc = np.random.randn(1000,3)
# real_pc = np.random.randn(1000,3)
# nerf_pc = np.load("generatedPointClouds/7710NeRF.npy")
# real_pc = np.load("generatedPointClouds/7710real.npy")
nerf_pc = np.load("generatedPointClouds/7750NeRF.npy")
real_pc = np.load("generatedPointClouds/7750real.npy")
# nerf_pc = np.load("generatedPointClouds/7950NeRF.npy")
# real_pc = np.load("generatedPointClouds/7950real.npy")
# nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput.npy")
# real_pc = np.load("generatedPointClouds/MaiCityReal.npy")

#train set
# nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_10.npy") 
# real_pc = np.load("generatedPointClouds/MaiCityReal_frame_10.npy")
# # #test set
# nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_v2_frame_0.npy") 
# real_pc = np.load("generatedPointClouds/MaiCityReal_v2_frame_0.npy")

# #remove ground plane
# real_pc = real_pc[real_pc[:,2] > -1]
# nerf_pc = nerf_pc[nerf_pc[:,2] > -1]

# #debug: create simple pc
# nerf_pc = np.array([[0,0,0], 
#                     [1,1,1]])
# real_pc = np.array([[0,0,0], 
#                     [0,0,0]])

#crop real PC down the same as NeRF output so bidirectional CD doesn't punish non-overlaps
# real_pc = real_pc[len(real_pc)//8:7*len(real_pc)//8,:]
real_pc = real_pc[len(real_pc)//8:31*len(real_pc)//32,:]


#remove points too close to origin
d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
good  = np.argwhere(d > 2.)[:,0]
nerf_pc = nerf_pc[good]
#remove points too close to origin
d = np.sqrt(np.sum(real_pc**2, axis = 1))
good  = np.argwhere(d > 2.)[:,0]
real_pc = real_pc[good]

#for NC only
# nerf_pc = np.flip(nerf_pc, axis = 0)

# #remove points out of range of sensor (only needed for Mai City)
# d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
# good = np.argwhere(d < 100)[:,0]
# nerf_pc = nerf_pc[good]

# nerf_pc = nerf_pc[::3,:]
# real_pc = real_pc[::3,:]
# nerf_pc = nerf_pc[:len(nerf_pc)//8,:]
# real_pc = real_pc[:len(real_pc)//8,:]

# spin_part = 2
# splits = 3 #8
# nerf_pc = nerf_pc[spin_part*len(real_pc)//splits:(spin_part+1)*len(nerf_pc)//splits,:]
# real_pc = real_pc[spin_part*len(real_pc)//splits:(spin_part+1)*len(real_pc)//splits,:]

print(np.shape(nerf_pc))
print(np.shape(real_pc))

(54393, 3)
(54019, 3)


In [71]:
# real_pc = np.linspace(0,9,10)[:,None]*np.array([1.,0.,0.])
# nerf_pc = np.linspace(0,9,10)[:,None]*np.array([1.,1.,0.]) + np.array([0.,1.,0])

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

disp.append(Points(real_pc[:,:], c = "red", r = 5))
disp.append(Points(nerf_pc[:,:], c = "blue", r = 5))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)


ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

# manually calcluate CD with torch

In [72]:
points1 = torch.tensor(nerf_pc.copy(), dtype=torch.float32)
points2 = torch.tensor(real_pc.copy(), dtype=torch.float32)

dists = torch.cdist(points1, points2)
# print(dists)
# Find the minimum distances
min_dists1 = dists.min(dim=1)[0]  # from points1 to points2
min_dists2 = dists.min(dim=0)[0]  # from points2 to points1
# print(min_dists1, min_dists1.mean())
# print(min_dists2, min_dists2.mean())

#chamfer distance
cd = (min_dists1.mean() + min_dists2.mean())/2
print("cd:", cd)

#Accuracy--
#"The accuracy metric is defined as the mean distance of points on the output 
#   mesh to their nearest neighbors on the ground truth mesh." --
accuracy = min_dists1.mean()
print("accuracy:", accuracy)

#completeness--
# The completeness metric is defined similarly, but in opposite direction.
completeness = min_dists2.mean()
print("completeness:", completeness)

cd: tensor(0.1537)
accuracy: tensor(0.1507)
completeness: tensor(0.1568)


In [74]:
(15.59+6.86)/2

11.225

In [ ]:
cd_hist = []
accuracy_hist = []
completeness_hist = []
good1_hist = []
good2_hist = []

for frame_idx in range(1): #25
    print("frame: ", frame_idx, "---------------")

    for spin_part in range(8):
#         nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_" + str(frame_idx) + ".npy")
#         real_pc = np.load("generatedPointClouds/MaiCityReal_frame_" + str(frame_idx) + ".npy")
        nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_v2_frame_" + str(frame_idx) + ".npy")
        real_pc = np.load("generatedPointClouds/MaiCityReal_v2_frame_" + str(frame_idx) + ".npy")

        # #remove ground plane
        # real_pc = real_pc[real_pc[:,2] > -1]
        # nerf_pc = nerf_pc[nerf_pc[:,2] > -1]

        # #debug: create simple pc
        # nerf_pc = np.array([[0,0,0], 
        #                     [1,1,1]])
        # real_pc = np.array([[0,0,0], 
        #                     [0,0,0]])

        #crop real PC down the same as NeRF output so bidirectional CD doesn't punish non-overlaps
        # real_pc = real_pc[len(real_pc)//8:7*len(real_pc)//8,:]
        # real_pc = real_pc[len(real_pc)//8:29*len(real_pc)//32,:]


        #remove points too close to origin
        d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
        good  = np.argwhere(d > 1)[:,0]
        nerf_pc = nerf_pc[good]

        # #remove points out of range of sensor (only needed for Mai City)
        # d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
        # good = np.argwhere(d < 100)[:,0]
        # nerf_pc = nerf_pc[good]

        nerf_pc = nerf_pc[spin_part*len(real_pc)//8:(spin_part+1)*len(nerf_pc)//8,:]
        real_pc = real_pc[spin_part*len(real_pc)//8:(spin_part+1)*len(real_pc)//8,:]

        points1 = torch.tensor(nerf_pc)
        points2 = torch.tensor(real_pc)

        dists = torch.cdist(points1, points2)
        # print(dists)
        # Find the minimum distances
        min_dists1 = dists.min(dim=1)[0]  # from points1 to points2
        min_dists2 = dists.min(dim=0)[0]  # from points2 to points1

    #     print(min_dists1, min_dists1.mean())
    #     print(min_dists2, min_dists2.mean())

        cd = (min_dists1.mean() + min_dists2.mean())/2        
        accuracy = min_dists2.mean() #TEST-- try flipping these values...
        completeness = min_dists1.mean()
        print("\n cd:", cd, "  accuracy: ", accuracy, " completeness:", completeness)

        cd_hist = np.append(cd_hist, cd)
        accuracy_hist = np.append(accuracy_hist, accuracy)
        completeness_hist = np.append(completeness_hist, completeness)
        
        good1 = np.where(min_dists1 < 0.1)[0]
        good1 = len(good1)/len(min_dists1)

        good2 = np.where(min_dists2 < 0.1)[0]
        good2 = len(good2)/len(min_dists2)

        good1_hist = np.append(good1_hist, good1)
        good2_hist = np.append(good2_hist, good2)
        print("good1, good2:", good1, good2)

In [ ]:
fig, ax = p.subplots()
# ax.plot(cd_hist, label = "CD")
# ax.plot(completeness_hist, label = "completeness")
# ax.plot(accuracy_hist, label = "accuracy")
ax.plot(good1_hist, label = "gh1")
ax.plot(good2_hist, label = "gh2")
ax.plot( np.mean(good1_hist**2)*np.ones([400]))

cutoff = 5000

print(np.mean(cd_hist[:cutoff]))
print(np.mean(accuracy_hist[:cutoff]))
print(np.mean(completeness_hist[:cutoff]))
print(np.mean((good1_hist)[:cutoff])) #too high to be F-score...?
print(np.mean((good2_hist)[:cutoff])) #too high to be F-score...?

In [ ]:
2*93*90.6/(93+90.6)

In [ ]:
#DEBUG ONLY-- look at data distribution witout tails

bidx = np.linspace(0,3,100)

fig, ax = p.subplots()
d1 = min_dists1.numpy()
d1_good_only = d1[d1 < 0.2]

d2 = min_dists2.numpy()
d2_good_only = d2[d2 < 0.2]

ax.hist(min_dists1, bins = bidx, alpha = 0.5);
ax.hist(d1_good, bins = bidx, alpha = 0.5);

print(np.mean(d1))
print(np.mean(d1_good_only))

print(np.mean(d2))
print(np.mean(d2_good_only))

# TEST: Get F-Score using torch cdists

In [ ]:
points1 = torch.tensor(nerf_pc)
points2 = torch.tensor(real_pc)

dists = torch.cdist(points1, points2)
# print(dists)
# Find the minimum distances
min_dists1 = dists.min(dim=1)[0]  # from points1 to points2
min_dists2 = dists.min(dim=0)[0]  # from points2 to points1
print(min_dists1, min_dists1.mean())

good1 = np.where(min_dists1 < 0.1)[0]
print(len(good1)/len(min_dists1))

good2 = np.where(min_dists2 < 0.1)[0]
print(len(good2)/len(min_dists2))

In [ ]:
real_pc = torch.tensor(real_pc)
nerf_pc = torch.tensor(nerf_pc)

if len(real_pc.shape) == 2:
    real_pc = real_pc.unsqueeze(0)  # Add batch dimension
if len(nerf_pc.shape) == 2:
    nerf_pc = nerf_pc.unsqueeze(0)  # Add batch dimension
    
# Convert to float
real_pc = real_pc.float()
nerf_pc = nerf_pc.float()
print(nerf_pc.shape)

chamferDist = ChamferDistance()
dist_forward = chamferDist(nerf_pc, real_pc)
print(dist_forward)
# print(dist_forward/ nerf_pc.shape[1])
# dist_bidirectional = chamferDist(nerf_pc, real_pc, bidirectional=True)
# print(dist_bidirectional)
# print(dist_bidirectional/ nerf_pc.shape[1])

# #TEST -- modify chamfer.py to produce output for calculating F-scores
# # dist_forward = chamferDist(nerf_pc, real_pc, reduction = 'fscore')
# dist_forward = chamferDist(nerf_pc, real_pc, reduction = 'fscore', reverse = True)
# # dist_forward is actually a SQUARED distance, so I need to take the sqrt first
# print(dist_forward)
# dist_forward = torch.sqrt(dist_forward)
# print(dist_forward)

# bad = np.argwhere(dist_forward > 0.2)

# print(np.shape(dist_forward)[1])
# print(np.shape(bad)[1])

# print(1 - np.shape(bad)[1] / np.shape(dist_forward)[1])

In [ ]:
#calculate map accuracy
good = np.argwhere(dist_forward < 0.2)[1,:]
print(len(good))
print(len(dist_forward[0]))
a = torch.mean(dist_forward[0][good])
# a = torch.mean(dist_forward[0])
print(a)

# print(dist_forward[0,:])

In [ ]:
tp = 0.9341 #true positives
fp = 1 - 0.9341 #false positives

#false negatives -- no estimated point close enough
fn = 1-0.9286 #use reverse nn distnaces here


# precision
p = tp/(tp+fp)

#recall
r = tp/(tp+fn)

f_score = 2*(p*r)/(p+r)
print(f_score)


In [ ]:
# print(savepc)
# print(pc1_aligned)
# print(dist_forward/ np.shape(nerf_pc)[1])

from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

#crop real PC down the same as NeRF output so bidirectional CD doesn't punish non-overlaps
# test = real_pc[0,len(real_pc[0,:,:])//8:7*len(real_pc[0,:,:])//8,:]
# print(np.shape(test))
# print(len(real_pc[0,:,:])//8)
# disp.append(Points(test, c = "red", r = 5))

disp.append(Points(real_pc[0,:,:], c = "red", r = 5))
disp.append(Points(nerf_pc[0,:,:], c = "blue", r = 5))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)


# calculate chamfer distance between NeRF output true distance for 25 frames Mai City Dataset

In [ ]:
CD_hist = []
chamferDist = ChamferDistance()

for frame_idx in range(25):
    print(frame_idx)
    nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_" + str(frame_idx) + ".npy")
    real_pc = np.load("generatedPointClouds/MaiCityReal_frame_" + str(frame_idx) + ".npy")

    #remove points too close to origin
    d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
    good  = np.argwhere(d > 1)[:,0]
    nerf_pc = nerf_pc[good]

    #remove points out of range of sensor (only needed for Mai City)
    d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
    good = np.argwhere(d < 100)[:,0]
    nerf_pc = nerf_pc[good]
    
#     print(np.shape(nerf_pc))
#     nerf_pc = nerf_pc[::3,:]
#     real_pc = real_pc[::3,:]
#     print(np.shape(nerf_pc))

    real_pc = torch.tensor(real_pc)
    nerf_pc = torch.tensor(nerf_pc)

    if len(real_pc.shape) == 2:
        real_pc = real_pc.unsqueeze(0)  # Add batch dimension
    if len(nerf_pc.shape) == 2:
        nerf_pc = nerf_pc.unsqueeze(0)  # Add batch dimension

    # Convert to float
    real_pc = real_pc.float()
    nerf_pc = nerf_pc.float()
    
#     dist_bidirectional = chamferDist(nerf_pc, real_pc, bidirectional=True)
#     dist_bidirectional /= nerf_pc.shape[1]
#     print(dist_bidirectional)
#     CD_hist = np.append(CD_hist, dist_bidirectional)


    dist_forward = chamferDist(nerf_pc, real_pc)
#     dist_forward /= nerf_pc.shape[1]
    print(dist_forward)
    CD_hist = np.append(CD_hist, dist_forward)

In [ ]:
from matplotlib import pyplot as p 

print(np.mean(CD_hist))
# np.save("generatedPointClouds/MaiCity_CD_hist.npy", CD_hist)
# CD_hist = np.load("generatedPointClouds/MaiCity_CD_hist.npy")
# print(CD_hist)
fig, ax = p.subplots()
ax.plot(CD_hist)

In [ ]:
from matplotlib import pyplot as p

# CD_hist_2 = CD_hist.copy()

fig, ax = p.subplots()
ax.set_title("Chamfer Distance for Mai City Dataset")
# ax.plot(CD_hist_15, label = "sample every 15th")
# ax.plot(CD_hist_10, label = "sample every 10th")
# ax.plot(CD_hist_5, label = "sample every 5th")
# ax.plot(CD_hist_2, label = "sample every 2nd")
ax.plot(CD_hist, label = "CD")
ax.set_xlabel("test frame index")
ax.set_ylabel("Chamfer Distance (cm)")
ax.legend(loc="best")

In [ ]:
#combine all NeRF outputs to single point cloud to render with open3d
combined_pc = np.zeros([0,3])
for frame_idx in range(50): #25
    print(frame_idx)
#     nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_" + str(frame_idx) + ".npy") #
    nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_velodyne_format_frame_" + str(frame_idx) + ".npy") #OS1 projections
#     nerf_pc = np.load("generatedPointClouds/MaiCityReal_frame_" + str(frame_idx) + ".npy") #DEBUG -- true scans

    #remove near zero points
#     nerf_pc = nerf_pc*200
    dists = np.sqrt(nerf_pc[:,0]**2 + nerf_pc[:,1]**2 + nerf_pc[:,2]**2)
    good = np.argwhere(dists > 2)[:,0]
#     print(good)
    nerf_pc = nerf_pc[good]

    nerf_pc = nerf_pc + np.array([frame_idx*.5,0.,0.])
#     nerf_pc = nerf_pc + np.array([frame_idx*1.,0.,0.])
    
    combined_pc = np.append(combined_pc, nerf_pc, axis = 0)

# print(np.shape(combined_pc))

# #crop to help with marching cubes??
# combined_pc = combined_pc[abs(combined_pc[:,0]) < 20]
# combined_pc = combined_pc[abs(combined_pc[:,1]) < 20]
# np.save("generatedPointClouds/CombinedMaiCity.npy", combined_pc)

import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_pc)
o3d.io.write_point_cloud("MaiCityNeRF.ply", pcd)

In [ ]:
#TEST -- load and crop HD Map to test marching cubes
import trimesh
pl = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-1stSection - mesh.ply'
raw_points = trimesh.load(pl).vertices
# points[:,2] = -points[:,2] #flip z
# points = points[::10,:] #downsample
points = raw_points.copy()

points = points[points[:,0]>0]
points = points[points[:,1]> - 2]

combined_pc = points
# np.save("generatedPointClouds/NCTestMap.npy", combined_pc)

import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_pc)
o3d.io.write_point_cloud("NCTestMap.ply", pcd)


In [ ]:
plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

disp.append(Points(combined_pc, c = "red", r = 2.5, alpha = 0.3))
disp.append(Points(raw_points, c = "black", r = 2.5, alpha = 0.1))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)


### Load 100 raw frames from Mai City sequence 01 and combine to a single point cloud

In [ ]:
# test = np.loadtxt("/media/derm/06EF-127D4/mai_city/txt/01/000000")

combined_pc = np.zeros([0,3])
for idx1 in range(100):
    print(idx1)
    pcfn = "/media/derm/06EF-127D4/mai_city/txt/01/" +  f"{idx1:06}"
    
    pc = np.loadtxt(pcfn)
    pc += np.array([1.*idx1, 0.,0.])
    pc = pc[::2]
    
    combined_pc = np.append(combined_pc, pc, axis = 0)

print(combined_pc)
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_pc)
o3d.io.write_point_cloud("MaiCityGTMap.ply", pcd)

# Calculate CD using pytorch3d chafer_distance() loss

In [ ]:
import pytorch3d

In [ ]:
%%bash
pip show torch